In [1]:
import os
import pandas as pd

In [2]:
data_dir = 'C:\\Users\\cindy\\Desktop\\Mini-Project-1-Safety-Analysis-of-Autonomous-Vehicles\\simulation_results\\'

### Import all the .csv data

##### 1.

In [3]:
all_data = []
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith('.csv'):
                file_path = os.path.join(folder_path, file)
                df = pd.read_csv(file_path)
                all_data.append((folder, file, df))
                # print(f"File: {file}")

for i, (folder, file, df) in enumerate(all_data):
    print(f"Data from folder '{folder}', file '{file}':\n", df.head(), "\n")
    print("="*50)

Data from folder 'route_highway_epoch24_clear-night_fi_ghost_cutin', file 'routes_fi_route_highway-0910_142625-data_fi_ghost_cutin-single_ctl.csv':
        ts  agent_id  throttle     steer  brake
0  542558         0       0.9 -0.003379    0.0
1  542559         0       0.9 -0.010903    0.0
2  542560         0       0.9 -0.009867    0.0
3  542561         0       0.9 -0.011255    0.0
4  542562         0       0.9  0.000766    0.0 

Data from folder 'route_highway_epoch24_clear-night_fi_ghost_cutin', file 'routes_fi_route_highway-0910_142625-data_fi_ghost_cutin-single_cvip.csv':
        ts  agent_id        cvip      cvip_x     cvip_y      cvip_z
0  542558         0  500.491189  198.767441 -95.832657 -499.819366
1  542559         0    5.595580  195.567444 -90.832657    0.100000
2  542560         0    5.592365  195.567444 -90.832657    0.095407
3  542561         0    5.589578  195.567444 -90.832657    0.084691
4  542562         0    5.587154  195.567444 -90.832657    0.069311 

Data from fol

In [4]:
# List the first 5 rows of the ctl.csv, cvip.csv, and traj.csv with the clear-night weather condition.
for folder, file, df in all_data:
    if ('ctl.csv' in file or 'cvip.csv' in file or 'traj.csv' in file) and 'clear-night' in folder:
        print(f"From file '{file}' (in folder '{folder}'):\n", df.head(), "\n")

From file 'routes_fi_route_highway-0910_142625-data_fi_ghost_cutin-single_ctl.csv' (in folder 'route_highway_epoch24_clear-night_fi_ghost_cutin'):
        ts  agent_id  throttle     steer  brake
0  542558         0       0.9 -0.003379    0.0
1  542559         0       0.9 -0.010903    0.0
2  542560         0       0.9 -0.009867    0.0
3  542561         0       0.9 -0.011255    0.0
4  542562         0       0.9  0.000766    0.0 

From file 'routes_fi_route_highway-0910_142625-data_fi_ghost_cutin-single_cvip.csv' (in folder 'route_highway_epoch24_clear-night_fi_ghost_cutin'):
        ts  agent_id        cvip      cvip_x     cvip_y      cvip_z
0  542558         0  500.491189  198.767441 -95.832657 -499.819366
1  542559         0    5.595580  195.567444 -90.832657    0.100000
2  542560         0    5.592365  195.567444 -90.832657    0.095407
3  542561         0    5.589578  195.567444 -90.832657    0.084691
4  542562         0    5.587154  195.567444 -90.832657    0.069311 

From file 'rout

##### 2.